In [ ]:
import pandas as pd

# Load event and position data
events = pd.read_csv("game_events_combined.csv")
ball_pos = pd.read_csv("ball_pos_combined.csv")

# Define function for infield bounds (adjust as needed)
def is_in_infield(x, y):
    return (-30 <= x <= 30) and (0 <= y <= 95)

# Get all 'hit into play' events
hits = events[events["event_code"] == 4][["game_str", "play_id", "timestamp"]].rename(columns={"timestamp": "hit_time"})

# Get all 'bounce' events
bounces = events[events["event_code"] == 16][["game_str", "play_id", "timestamp"]].rename(columns={"timestamp": "bounce_time"})

# Get first bounce per play
first_bounce = bounces.sort_values("bounce_time").drop_duplicates(subset=["game_str", "play_id"], keep="first")

# Merge hits with first bounce
merged = hits.merge(first_bounce, on=["game_str", "play_id"], how="inner")
merged = merged[merged["bounce_time"] > merged["hit_time"]]

# Join with ball_pos to get coordinates of the bounce
bounce_with_pos = merged.merge(
    ball_pos[["game_str", "play_id", "timestamp", "ball_position_x", "ball_position_y"]],
    left_on=["game_str", "play_id", "bounce_time"],
    right_on=["game_str", "play_id", "timestamp"],
    how="left"
)

# Check if the bounce was in the infield
bounce_with_pos["infield_bounce"] = bounce_with_pos.apply(
    lambda row: is_in_infield(row["ball_position_x"], row["ball_position_y"]),
    axis=1
)

# Final result — hits where the first bounce was in the infield
infield_hits = bounce_with_pos[bounce_with_pos["infield_bounce"]].drop_duplicates(subset=["game_str", "play_id"])


In [19]:
print(infield_hits)
infield_hits.to_csv('grounders.csv')


             game_str  play_id  hit_time  bounce_time   timestamp  \
1     y1_d069_ACN_QEA       19    632253       632453    632453.0   
4     y1_d069_ACN_QEA       46   1728503      1728603   1728603.0   
5     y1_d069_ACN_QEA       49   1789903      1790003   1790003.0   
6     y1_d069_ACN_QEA       67   2499503      2499553   2499553.0   
7     y1_d069_ACN_QEA       78   2811503      2811853   2811853.0   
...               ...      ...       ...          ...         ...   
8860  y1_d058_WZR_YJD      314  11358949     11359279  11359279.0   
8862  y1_d058_WZR_YJD      320  11506228     11506558  11506558.0   
8863  y1_d058_WZR_YJD      324  11750494     11751187  11751187.0   
8864  y1_d058_WZR_YJD      325  11794286     11794385  11794385.0   
8865  y1_d058_WZR_YJD      330  12025847     12025946  12025946.0   

      ball_position_x  ball_position_y  infield_bounce  
1            20.21193          31.1913            True  
4             2.94990          15.8268            True  
